In [3]:
#Import innitial dependencies
import warnings
warnings.simplefilter('ignore')

#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
# Read the csv file into a pandas DataFrame
dataset = pd.read_csv('../data/ModelData_1.csv')
dataset.head()
dataset.tail()

,Servings/s,Calories/s,Fat/s,Sat_Fat/s,Trans_Fat/s,Mono_Fat/s,Poly_Fat/s,Carbs/s,Fiber/s,Sugars/s,...,B6/s,Folate_eq/s,Folate_food/s,B12/s,Vit_D/s,Vit_E/s,Vit_K/s,Water/s,Healthy,HealthyLabel
1844,1.0,885.555968,53.018457,21.638624,0.465476,22.651813,3.703198,41.120828,3.092770,5.914407,...,1.090329,151.388358,73.868358,2.002937,8.520000,2.797755,43.488101,288.697325,0,unhealthy
1845,1.0,1199.745198,52.064223,21.695331,0.055055,20.987810,6.193580,132.531893,9.903497,10.157808,...,0.435342,698.420402,95.898856,0.855400,20.160000,3.487766,9.842727,299.033738,0,unhealthy
1846,1.0,575.813376,29.716701,17.255565,0.668482,7.652011,1.418145,48.349441,2.459127,2.626872,...,0.156702,34.096367,34.096367,0.675327,18.282771,0.563872,2.312221,120.323226,0,unhealthy
1847,1.0,223.868965,8.640837,3.993991,0.006804,2.284186,1.262668,26.627345,5.083299,11.188909,...,0.352999,63.463372,40.945372,0.227013,4.082331,3.523929,25.027532,204.756052,0,unhealthy
1848,1.0,453.053030,29.118689,15.106494,0.082530,8.686451,1.648327,34.804224,2.571149,5.466125,...,0.261409,40.777834,40.777834,0.306823,18.406219,0.925100,12.385577,137.497637,0,unhealthy


In [51]:
# Assign X (data) and y (target) - And review 
X = dataset.drop("HealthyLabel", axis=1)
y = dataset["HealthyLabel"]
print(X.shape, y.shape)

(1849, 33) (1849,)


In [52]:
# Split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [53]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend

In [55]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [56]:
y_train_categorical

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [57]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('modelD1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modelD1.h5")
print("Loaded modelD1 from disk")

Loaded modelD1 from disk


In [58]:
loaded_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [1]:
#model_loss, model_accuracy = loaded_model.evaluate(
 #   X_test_scaled, y_test_categorical, verbose=2)
#print(
 #   f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [62]:
encoded_predictions = loaded_model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [63]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['unhealthy' 'healthy' 'healthy' 'unhealthy' 'healthy']
Actual Labels: ['unhealthy', 'healthy', 'healthy', 'unhealthy', 'healthy']
